In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('C:/Users/Dell/Downloads/ANN_DATA.csv')
df.set_index('Date', inplace=True)

In [ ]:
df.head()

##Data Preprocessing 

In [ ]:
len(df.values)

In [ ]:
def convert2matrix(data_arr, look_back):
 X, Y =[], []
 for i in range(len(data_arr)-look_back):
  d=i+look_back  
  X.append(data_arr[i:d,0])
  Y.append(data_arr[d,0])
 return np.array(X), np.array(Y)

train_size = 100
train, test =df.values[0:train_size,:],df.values[train_size:len(df.values),:]
look_back = 7# setup look_back window 

#converting dataset into right shape in order to input into the DNN
trainX, trainY = convert2matrix(train, look_back)
testX, testY = convert2matrix(test, look_back)

In [ ]:
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

##  Define neural network shape and compile model

In [ ]:

def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(21, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

In [ ]:
model=model_dnn(look_back)
history=model.fit(trainX,trainY, 
                  epochs=100, batch_size=35, verbose=1, 
                  validation_data=(testX,testY),
                  callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
                  shuffle=False)

In [ ]:
 def model_loss(history):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Test Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show();

In [ ]:
train_score = model.evaluate(trainX, trainY, verbose=0)
print('Train Root Mean Squared Error(RMSE): %.2f; Train Mean Absolute Error(MAE) : %.2f ' 
% (np.sqrt(train_score[1]), train_score[2]))
test_score = model.evaluate(testX, testY, verbose=0)
print('Test Root Mean Squared Error(RMSE): %.2f; Test Mean Absolute Error(MAE) : %.2f ' 
% (np.sqrt(test_score[1]), test_score[2]))
model_loss(history)

In [ ]:
def prediction_plot(testY, test_predict):
  len_prediction=[x for x in range(len(testY))]
  plt.figure(figsize=(8,4))
  plt.plot(len_prediction, testY, marker='.', label="actual")
  plt.plot(len_prediction, test_predict, 'r', label="prediction")
  plt.tight_layout()
  sns.despine(top=True)
  plt.subplots_adjust(left=0.07)
  plt.ylabel('Temp', size=15)
  plt.xlabel('Time step', size=15)
  plt.legend(fontsize=15)
  plt.show();

In [ ]:
train_predict = model.predict(trainX)
prediction_plot(trainY, train_predict)

In [ ]:
Temp = pd.DataFrame(columns=['actual','predicted'])
Temp['actual'] = list(trainY.reshape(93,))
Temp['predicted'] = list(train_predict.reshape(93,))

In [ ]:
Temp.to_csv('Final_predicted.csv')

In [ ]:
plt.plot(Temp['actual'], Temp['predicted'], 'o', color='black');
plt.xlabel('Actual')
plt.ylabel('predicted')